# CS4243 - Lab Session 6

Computer Vision & Pattern Recognition

Week 9, Mon 17 Oct, AY 2023/24

Author: Dr. Amirhassan MONAJEMI. Modified by: Lingdong KONG

## Welcome! 👋

This notebook contains the tutorials for the `sixth` lab sessions. The following materials are covered:

- Part 1: Optical Flow

Let's get started!

<hr>

# Part 1: Optical Flow

#### Goal of this section:
- Learn how to track feature points using the *Lucas-Kanade algorithm* in the `lucas_kanade_point_tracking` function.
- Learn how to draw the tracked points and the motion vectors on each frame.

    Note:
    
    - The tracking continues until you press the `'Esc'` key to exit the application.
    - Make sure that the path to your video file is correct. 

  Extra:
    
    - You can configure the code to use your camera as the video source.
    - You might need to adjust the parameters for good feature point detection (e.g., `cv2.goodFeaturesToTrack`) to suit your specific tracking needs.

We start by capturing video frames from a video file.


#### Videos for testing: 
- Example 1: `vtest.avi`
- Example 2: `10142.mp4`
- Example 3: `10231.mp4`
- Example 4: `10236.mp4`

You can download these videos from Canvas: CS4243 -> Files -> Python_notebooks -> set5 -> `OF_samples.zip`

#### Procedures:
1. Run the code using a few test videos (as listed above).
2. Understand the role of the parameters and change them accordingly to comments.
3. Try to make it better again based on the comments.
4. Answer the questions.

In [1]:
# importing necessary libraries 
import cv2
import numpy as np

**Main Function** (`lucas_kanade_point_tracking`)

- Input parameters:
    - Two frames, `prev_frame` and `curr_frame`, and an array of `prev_points` coordinations.
- Returns:
    - The updated `prev_points` and the new `next_points` arrays.

In [2]:
def lucas_kanade_point_tracking(prev_frame, curr_frame, prev_points):
    
    lk_params = dict(
        winSize=(15, 15),
        maxLevel=2,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03),
    )

    # Step 1: calculate optical flow using Lucas-Kanade method
    next_points, status, _ = cv2.calcOpticalFlowPyrLK(
        prev_frame, curr_frame, prev_points, None, **lk_params
    )
    status = status.ravel()
    
    # Step 2: filter out points with status = 1 (successfully tracked)
    prev_points = prev_points[status == 1]
    next_points = next_points[status == 1]

    return prev_points, next_points

**Questions:**

> **Q1** - What are the types and shapes of those parameters?
> - **Answer:**
> - `prev_frame`: The type is `numpy.ndarray` and the shape is (720, 1280, 3).
> - `curr_frame`: The type is `numpy.ndarray` and the shape is (720, 1280, 3).
> - `prev_points`: The type is `numpy.ndarray` and the shape is (100, 1, 2).
    

> **Q2** - What are the shapes of those returned arrays? Any changes in `prev_points`?
> - **Answer:**
> - `prev_points`: The shape is (100, 1, 2).
> - `next_points`: The shape is (100, 1, 2).
    

> **Q3** - What are the parameters for the Lucas-Kanade optical flow method? what do they do?
> - **Answer:**
> - `winSize`: Size of the search window at each pyramid level.
> - `maxLevel`: 0-based maximal pyramid level number; if set to 0, pyramids are not used (single level), if set to 1, two levels are used, and so on.
> - `criteria`: Parameter specifying the termination criteria of the iterative search algorithm (after the specified maximum number of iterations `criteria.maxCount` or when the search window moves by less than `criteria.epsilon`).
    
 
> **Q4** - What are the criteria? how to finish an iterative algorithm?
> - **Answer:**
> - `cv2.TERM_CRITERIA_EPS`: Termination criteria for the iterative search algorithm, which indicates that the algorithm should stop when the specified accuracy or epsilon is reached.
> - `cv2.TERM_CRITERIA_COUNT`: Termination criteria for the iterative search algorithm, which indicates that the algorithm should stop after the specified number of iterations.
> - `cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT`: Termination criteria for the iterative search algorithm, which indicates that the algorithm should stop when the specified accuracy or epsilon is reached, or when the search window moves by less than the specified number of iterations.
    

**References:**

- Source 1: https://docs.opencv.org/3.4/d4/dee/tutorial_optical_flow.html
- Source 2: https://www.geeksforgeeks.org/python-opencv-optical-flow-with-lucas-kanade-method

In [3]:
# load the video
cap = cv2.VideoCapture('/Users/tianze/cs4243_lab/Python_notebooks/set5/OF_samples/vtest.avi')

Learn more about the `VideoCapture` function of OpenCV. 

**Questions:**

> **Q5** - Do we need other parameters here?
> - **Answer:**
    

> **Q6** - How can we read a live video stream?
> - **Answer:**


**References:**
- Source 3: https://docs.opencv.org/4.x/dd/d43/tutorial_py_video_display.html

In [4]:
# read the first frame
ret, prev_frame = cap.read()

In [5]:
print(ret)

True


In [6]:
print(prev_frame.shape)

(576, 768, 3)


**Questions:**

> **Q7** - What does the `ret` parameter do here?
> - **Answer:**
> - `ret`: A boolean value indicating whether the frame is read correctly or not.
    

In [7]:
# convert the frame to gray level
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

Define initial points to track (e.g., using `cv2.goodFeaturesToTrack`)
- We can have any function which returns an array of the [x, y] coordination pairs here.

Input Parameters:
- `prev_gray`: the frame that used to extract feature points or anchor points from.
- `maxCorners`: the maximum number of points to detect.

In [8]:
prev_points = cv2.goodFeaturesToTrack(
    prev_gray,
    maxCorners=100,
    qualityLevel=0.3,
    minDistance=7,
)

**Questions:**

> **Q8** - What if changing `maxCorners` to `4`?
> - **Answer:**
> - The number of points to detect will be 4.
> - The points will be the four corners of the image.
    

> **Q9** - What if changing `qualityLevel` to `0.9` and `0.01`?
> - **Answer:**
> - The number of points to detect will be 100.
> - The points will be the four corners of the image.
    

> **Q10** - The `minDistance` parameter is the minimum distance between detected feature points in pixel. What if changing it to `1` and `50`?
> - **Answer:**
    

**References:**
- Source 4: https://docs.opencv.org/3.4/d4/d8c/tutorial_py_shi_tomasi.html 
- Source 5: https://theailearner.com/tag/cv2-goodfeaturestotrack

In [9]:
while True:
    
    ret, curr_frame = cap.read()
    
    if not ret:
        break

    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

    # track points using Lucas-Kanade method
    prev_points, next_points = lucas_kanade_point_tracking(prev_gray, curr_gray, prev_points)

    # draw tracks on the current frame
    for i, (prev, next) in enumerate(zip(prev_points, next_points)):
        x1, y1 = prev.ravel()
        x2, y2 = next.ravel()
        cv2.line(curr_frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
        cv2.circle(curr_frame, (int(x2), int(y2)), 5, (0, 255, 0), -1) 
        
    cv2.namedWindow("Optical Flow", cv2.WINDOW_NORMAL)
    cv2.imshow('Optical Flow', curr_frame)
    
    # press 'Esc' to exit
    if cv2.waitKey(0) & 0xFF == 27:
        break

**Questions:**

> **Q11** - How can we break the `while` loop? why anded with `0xFF`?
> - **Answer:**
> - We can break the `while` loop by pressing the `Esc` key.
    


In [10]:
cap.release()
cv2.destroyAllWindows()

<hr>

## Next:

We have seen an example using video `10231.mp4`. You might want to explore other videos in this tutorial:

- Example 1: `vtest.avi`
- Example 2: `10142.mp4`
- Example 3: `10231.mp4` (Completed)
- Example 4: `10236.mp4`

### 🎉 Congratulations! You have finished this lab tutorial.